# Plot CMIP6 ensemble for GRDC basins
Make visual of ~110 simulations from the CMIP6 archive for SSP2-4.5.  Compare these with the 12 simulations chosen for standard glacier model forcing.

29 Jan 2025 | EHU

- 6 Feb: Update rolling window length to 20 rather than 15 years, for more direct comparison with the 20-year-long baseline period.  Add some single-basin examples with builds to show the full ensemble and then highlight the glacier model forcing
- 25 Jun: Re-size the 75-basin figure for a journal page layout.
- 12 Aug: Summarize in strip plot of spread and scatter plot of means
- 10 Dec: Bar and whiskers: GEM-forcing spread as bar and GCM spread as whiskers.  Correct small error in precip unit conversion (calling wrong basin area); no visible effect on results.

In [ ]:
import datetime
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [ ]:
## Define the filepath streamed from OneDrive
fpath = '/Users/eultee/Library/CloudStorage/OneDrive-NASA/Documents/Runoff/'

f_PCP = fpath+'CMIP-data/PrecipLizz/'
f_T = fpath+'CMIP-data/TempLizz/'

In [ ]:
## Get the basin list in the order Sloan usually uses
path_to_area = fpath+'BasinArea.mat' ## uploaded by Sloan
BasinAreas = loadmat(path_to_area)
#Creating indices a dataframe would use
basin_areas = BasinAreas['BasinArea']
basin_names = BasinAreas['BasinNam']
basin_name_list = [name[1][0] for name in basin_names]

revised_names = {'ISSYK-KUL': 'YSYK-KOL', 'LAGARFLIOT': 'LAGARFLJOT'}.get ## revise to align Sloan's names with Finn's spelling
basin_name_list = [revised_names(n,n) for n in basin_name_list]

basin_names=basin_name_list ## just to make sure to get them all

TotalBasinAreas = pd.DataFrame({'Basin Area': basin_areas.squeeze()}, index=basin_name_list).sort_index() ## required for unit conversion later

## Load in all GCMs from the ensemble

In [ ]:
file_pattern = os.path.join(f_PCP, f"*.txt")
file_list = glob.glob(file_pattern)

P_by_model = []

for j, file in enumerate(file_list):
    P_by_model.append(np.loadtxt(file))
    
np.shape(P_by_model)

In [ ]:
P_by_basin_all = {b: {} for b in basin_name_list}

for k in range(len(file_list)):
    for i,b in enumerate(basin_name_list):
        P_by_basin_all[b][k] = P_by_model[k][:,i]

In [ ]:
P_by_basin_all.keys()

## Plot all basins against a baseline

In [ ]:
rng = pd.date_range('2000-01-01', periods=1212, freq='ME')

In [ ]:
fig, axs = plt.subplots(15,5, figsize=(13,16), sharex=True)
for j,b in enumerate(basin_name_list):
    ax = axs.ravel()[j]
    ax.axhline(0, color='k', ls=':', lw=0.5)
    
    basin_label = ' '.join([s.capitalize() for s in b.split(' ')]) ## manage multi-word basin names
    
    for m in range(len(file_list)): ## split by model run
        this_precip = pd.Series(P_by_basin_all[b][m])
        this_precip.index=rng

        this_precip_dailyavg = this_precip / this_precip.index.days_in_month
        ## handy days_in_month utility
        this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)

        hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        ax.plot((this_precip_mmday.resample('YE').mean()-hist_mean).rolling(window=20).mean()/(hist_mean*0.01), ##
               color='grey', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top

    ax.set(xlim=(datetime.datetime(1999,12,31),datetime.datetime(2100,12,31)),
          ylim=[-20,20])
    ax.text(0.1, 0.8, basin_label, transform=ax.transAxes)

axs[0,4].legend(bbox_to_anchor=(1.05, 1.0), ncol=1)
axs[14,2].set_xlabel('Year', fontsize=14) ## use a direct axis label so that suptitle is not so far away from the axes
for k in range(len(axs[14])):
    axs[14,k].set(xticks=pd.date_range(datetime.datetime(2000,1,1), periods=6, freq='20YS').tolist(),
                  xticklabels=['2000','','','','', '2100'])
    axs[14,k].tick_params(axis='x', rotation=45)

# fig.supxlabel('Month', fontsize=14)
fig.suptitle('Basin precip % change (20-year rolling mean, rel. to 2000-2020) \n from GCMs used to force glacier models',
            fontsize=14);
# fig.supxlabel('Year', fontsize=14);
fig.supylabel('Basin precip change [% 2000-2020]', fontsize=14);
plt.tight_layout()

## Highlight the models included in glacier model forcing

In [ ]:
f_PCP_gems = f_PCP+'GlacierModel/'

file_pattern_gems = os.path.join(f_PCP_gems, f"*.txt")
file_list_gems = glob.glob(file_pattern_gems)

P_by_model_gems = []

for j, file in enumerate(file_list_gems):
    P_by_model_gems.append(np.loadtxt(file))
    
np.shape(P_by_model_gems)

In [ ]:
P_by_basin_gems = {b: {} for b in basin_name_list}

for k in range(len(file_list_gems)):
    for i,b in enumerate(basin_name_list):
        P_by_basin_gems[b][k] = P_by_model_gems[k][:,i]

In [ ]:
import matplotlib.patches as mpatches


fig, axs = plt.subplots(11,7, figsize=(10.5,12), sharex=True, sharey=True)
for j,b in enumerate(basin_name_list):
    ax = axs.ravel()[2::][j]
    ax.axhline(0, color='k', ls=':', lw=0.5)
    
    basin_label = ' '.join([s.capitalize() for s in b.split(' ')]) ## manage multi-word basin names
    if basin_label=='Lake Balkhash':
        basin_label = 'Balkhash'
    elif basin_label=='Jokulsa A Fjollum':
        basin_label = 'Jokulsa A Fj.'
    else:
        pass
    
    for m in range(len(file_list)): ## split by model run
        this_precip = pd.Series(P_by_basin_all[b][m])
        this_precip.index=rng

        this_precip_dailyavg = this_precip / this_precip.index.days_in_month
        ## handy days_in_month utility
        this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)

        hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        ax.plot((this_precip_mmday.resample('YE').mean()-hist_mean).rolling(window=20).mean()/(hist_mean*0.01), ##
               color='grey', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top
    
    for n in range(len(file_list_gems)): ## highlight the ones used for GEM forcing
        this_precip = pd.Series(P_by_basin_gems[b][n])
        this_precip.index=rng

        this_precip_dailyavg = this_precip / this_precip.index.days_in_month
        ## handy days_in_month utility
        this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)

        hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        ax.plot((this_precip_mmday.resample('YE').mean()-hist_mean).rolling(window=20).mean()/(hist_mean*0.01), ##
               color='blue', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top

    ax.set(xlim=(datetime.datetime(2020,12,31),datetime.datetime(2100,12,31)),
          ylim=[-25,25])
    if basin_label in ['Huasco', 'Copiapo', 'Rapel']: ## address hard-to-read cases
        text_color='white'
        ax.text(0.4, 0.05, basin_label, color=text_color, transform=ax.transAxes)
    else:
        text_color='black'
        ax.text(0.05, 0.05, basin_label, color=text_color, transform=ax.transAxes)

## legend
patches=[mpatches.Patch(color='grey', label='CMIP6 ensemble'),
        mpatches.Patch(color='blue', label='Used to force GEMs')]
# axs[0,4].legend(handles=patches, bbox_to_anchor=(1.05, 1.0), ncol=1)
axs[0,2].legend(handles=patches, bbox_to_anchor=(-0.2, 1.0), ncol=1)
## put legend in the empty space at the top left

[fig.delaxes(ax) for ax in axs.flatten() if not ax.has_data()] ## remove "leftover" axes with no data


axs[10,3].set_xlabel('Year', fontsize=14) ## use a direct axis label so that suptitle is not so far away from the axes
for k in range(len(axs[10])):
    axs[10,k].set(xticks=pd.date_range(datetime.datetime(2020,1,1), periods=5, freq='20YS').tolist(),
                  xticklabels=['2020','','','', '2100'])
    axs[10,k].tick_params(axis='x', rotation=45)

# fig.supxlabel('Month', fontsize=14)
# fig.suptitle('Basin precip % change (20-year rolling mean, rel. to 2000-2020) \n from GCMs used to force glacier models',
#             fontsize=14);
# fig.supxlabel('Year', fontsize=14);
fig.supylabel('Basin precip change [% 2000-2020]', fontsize=14, x=0.05);
plt.tight_layout()

# plt.savefig('/Users/eultee/Downloads/20250625-RfIntercomp-AllBasins-P-All.png', dpi=300)
plt.show()

---
## Plot temperature in a similar way

In [ ]:
file_pattern_T = os.path.join(f_T, f"*.txt")
file_list_T = glob.glob(file_pattern_T)

T_by_model = []

for j, file in enumerate(file_list_T):
    T_by_model.append(np.loadtxt(file))
    
# np.shape(T_by_model)

f_T_gems = f_T+'GlacierModel/'

file_pattern_T_gems = os.path.join(f_T_gems, f"*.txt")
file_list_T_gems = glob.glob(file_pattern_T_gems)

T_by_model_gems = []

for j, file in enumerate(file_list_T_gems):
    T_by_model_gems.append(np.loadtxt(file))

In [ ]:
T_by_basin_all = {b: {} for b in basin_name_list}
T_by_basin_gems = {b: {} for b in basin_name_list}

for k in range(len(file_list_T)):
    for i,b in enumerate(basin_name_list):
        T_by_basin_all[b][k] = T_by_model[k][:,i]

for p in range(len(file_list_T_gems)):
    for j,b in enumerate(basin_name_list):
        T_by_basin_gems[b][p] = T_by_model_gems[p][:,j]

In [ ]:
fig, axs = plt.subplots(11,7, figsize=(10.5,12), sharex=True, sharey=True)
for j,b in enumerate(basin_name_list):
    ax = axs.ravel()[2::][j]
    ax.axhline(0, color='k', ls=':', lw=0.5)
    
    basin_label = ' '.join([s.capitalize() for s in b.split(' ')]) ## manage multi-word basin names
    if basin_label=='Lake Balkhash':
        basin_label = 'Balkhash'
    elif basin_label=='Jokulsa A Fjollum':
        basin_label = 'Jokulsa A Fj.'
    else:
        pass
    
    for m in range(len(file_list_T)): ## split by model run
        this_temp = pd.Series(T_by_basin_all[b][m])
        this_temp.index=rng

        # this_temp_dailyavg = this_temp / this_temp.index.days_in_month
        # ## handy days_in_month utility
        # this_temp_mmday = this_temp_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)

        hist_mean = this_temp['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        ax.plot((this_temp.resample('YE').mean()-hist_mean).rolling(window=20).mean(), ##
               color='grey', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top
    
    for n in range(len(file_list_T_gems)): ## highlight the ones used for GEM forcing
        this_temp = pd.Series(T_by_basin_gems[b][n])
        this_temp.index=rng

        # this_temp_dailyavg = this_temp / this_temp.index.days_in_month
        # ## handy days_in_month utility
        # this_temp_mmday = this_temp_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)

        hist_mean = this_temp['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        ax.plot((this_temp.resample('YE').mean()-hist_mean).rolling(window=20).mean(), ##
               color='blue', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top

    ax.set(xlim=(datetime.datetime(2020,12,31),datetime.datetime(2100,12,31)),
          ylim=[-1,5] ## Set to values in K rather than percent change
          )
    ax.text(0.05, 0.8, basin_label, transform=ax.transAxes)

## legend
patches=[mpatches.Patch(color='grey', label='CMIP6 ensemble'),
        mpatches.Patch(color='blue', label='Used to force GEMs')]
# axs[0,4].legend(handles=patches, bbox_to_anchor=(1.05, 1.0), ncol=1)
axs[0,2].legend(handles=patches, bbox_to_anchor=(-0.2, 1.0), ncol=1)
## put legend in the empty space at the top left

[fig.delaxes(ax) for ax in axs.flatten() if not ax.has_data()] ## remove "leftover" axes with no data

axs[10,3].set_xlabel('Year', fontsize=14) ## use a direct axis label so that suptitle is not so far away from the axes
for k in range(len(axs[10])):
    axs[10,k].set(xticks=pd.date_range(datetime.datetime(2020,1,1), periods=5, freq='20YS').tolist(),
                  xticklabels=['2020','','','', '2100'])
    axs[10,k].tick_params(axis='x', rotation=45)

# fig.supxlabel('Month', fontsize=14)
# fig.suptitle('Basin temp anomaly (20-year rolling mean, rel. to 2000-2020) \n from GCMs used to force glacier models',
#             fontsize=14);
# fig.supxlabel('Year', fontsize=14);
fig.supylabel('Basin temp anomaly [K]', fontsize=14, x=0.05);
plt.tight_layout()

# plt.savefig('/Users/eultee/Downloads/20250625-RfIntercomp-AllBasins-T-All.png', dpi=300)
plt.show()


## Single basin examples, with builds
Plot a single basin to introduce this style of plot during a talk.  Choose Rhone (easy), Indus, Santa?

In [ ]:
example_basins = ['RHONE']

from matplotlib import rcParams
rcParams["axes.labelsize"] = 14

## as of 27 Mar 25: adding custom figsize for proposal.  Remove to have default shape shown in talks.
for b in example_basins:
    fig, axs = plt.subplots(2,1, figsize=(5,7), sharex=True) #stack vertically to make sensible shared x

    ## Plot temperature on first plot
    axs[0].axhline(0, color='k', ls=':', lw=0.5)
    
    basin_label = ' '.join([s.capitalize() for s in b.split(' ')]) ## manage multi-word basin names
    
    for m in range(len(file_list_T)): ## split by model run
        this_temp = pd.Series(T_by_basin_all[b][m])
        this_temp.index=rng
        hist_mean = this_temp['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        axs[0].plot((this_temp.resample('YE').mean()-hist_mean).rolling(window=20).mean(), ##
               color='grey', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top
    
    for n in range(len(file_list_T_gems)): ## highlight the ones used for GEM forcing
        this_temp = pd.Series(T_by_basin_gems[b][n])
        this_temp.index=rng
        hist_mean = this_temp['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        axs[0].plot((this_temp.resample('YE').mean()-hist_mean).rolling(window=20).mean(), ##
               color='blue', alpha=0.5)
        
    axs[0].set(xlim=(datetime.datetime(2020,12,31),datetime.datetime(2100,12,31)),
          ylim=[-1,5], ## Set to values in K rather than percent change
               ylabel = 'Temp change [K]'
          )
    # axs[0].text(0.05, 0.8, basin_label, transform=axs[0].transAxes)
    
    ## Plot precipitation on next one
    for m in range(len(file_list)): ## split by model run
        this_precip = pd.Series(P_by_basin_all[b][m])
        this_precip.index=rng

        this_precip_dailyavg = this_precip / this_precip.index.days_in_month
        ## handy days_in_month utility
        this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)

        hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        axs[1].plot((this_precip_mmday.resample('YE').mean()-hist_mean).rolling(window=20).mean()/(hist_mean*0.01), ##
               color='grey', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top
    
    for n in range(len(file_list_gems)): ## highlight the ones used for GEM forcing
        this_precip = pd.Series(P_by_basin_gems[b][n])
        this_precip.index=rng

        this_precip_dailyavg = this_precip / this_precip.index.days_in_month
        ## handy days_in_month utility
        this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)

        hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        axs[1].plot((this_precip_mmday.resample('YE').mean()-hist_mean).rolling(window=20).mean()/(hist_mean*0.01), ##
               color='blue', alpha=0.5)
    
    axs[1].axhline(0, color='k', lw=0.5)
    axs[1].set(xlim=(datetime.datetime(2020,12,31),datetime.datetime(2100,12,31)),
               xticks=pd.date_range(datetime.datetime(2020,1,1), periods=5, freq='20YS').tolist(),
               xticklabels=['2020','2040','2060','2080', '2100'],
               ylim=[-25,25],
               ylabel='Precip change[%]')
    
    for ax in axs:
        ax.tick_params(axis='both', labelsize=12)  
    
    plt.tight_layout()
    plt.show()

Build just one, for example Indus precip.

In [ ]:
highlight_basin = 'INDUS'
basin_label = ' '.join([s.capitalize() for s in highlight_basin.split(' ')]) ## manage multi-word basin names
to_plot = 'P' # choose T or P


fig, ax = plt.subplots(figsize=(6,4))
ax.axhline(0, color='k', ls=':', lw=0.5)
    
if to_plot=='T': ## plot temp
    for m in range(len(file_list_T)): ## split by model run
        this_temp = pd.Series(T_by_basin_all[highlight_basin][m])
        this_temp.index=rng
        hist_mean = this_temp['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        ax.plot((this_temp.resample('YE').mean()-hist_mean).rolling(window=20).mean(), ##
               color='grey', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top

    # for n in range(len(file_list_T_gems)): ## highlight the ones used for GEM forcing
    #     this_temp = pd.Series(T_by_basin_gems[highlight_basin][n])
    #     this_temp.index=rng
    #     hist_mean = this_temp['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
    #     ax.plot((this_temp.resample('YE').mean()-hist_mean).rolling(window=20).mean(), ##
    #            color='blue', alpha=0.5)
    
    ax.set(xlim=(datetime.datetime(2020,12,31),datetime.datetime(2100,12,31)),
           xticks=pd.date_range(datetime.datetime(2020,1,1), periods=5, freq='20YS').tolist(),
           xticklabels=['2020','2040','2060','2080', '2100'],
      ylim=[-1,5], ## Set to values in K rather than percent change
           ylabel = 'Temp change [K]'
      )

elif to_plot=='P': ## plot precip
    for m in range(len(file_list)): ## split by model run
        this_precip = pd.Series(P_by_basin_all[highlight_basin][m])
        this_precip.index=rng

        this_precip_dailyavg = this_precip / this_precip.index.days_in_month
        ## handy days_in_month utility
        this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[highlight_basin].squeeze()*1e6)

        hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        ax.plot((this_precip_mmday.resample('YE').mean()-hist_mean).rolling(window=20).mean()/(hist_mean*0.01), ##
               color='grey', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top

#     for n in range(len(file_list_gems)): ## highlight the ones used for GEM forcing
#         this_precip = pd.Series(P_by_basin_gems[highlight_basin][n])
#         this_precip.index=rng

#         this_precip_dailyavg = this_precip / this_precip.index.days_in_month
#         ## handy days_in_month utility
#         this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[highlight_basin].squeeze()*1e6)

#         hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
#         ax.plot((this_precip_mmday.resample('YE').mean()-hist_mean).rolling(window=20).mean()/(hist_mean*0.01), ##
#                color='blue', alpha=0.5)
    
    ax.set(xlim=(datetime.datetime(2020,12,31),datetime.datetime(2100,12,31)),
           xticks=pd.date_range(datetime.datetime(2020,1,1), periods=5, freq='20YS').tolist(),
           xticklabels=['2020','2040','2060','2080', '2100'],
           ylim=[-30,71],
           ylabel='Precip change[%]')
else:
    print('Choose either T or P to plot')

ax.text(0.05, 0.9, basin_label, transform=ax.transAxes)

# patches=[mpatches.Patch(color='k', label='CMIP6 ensemble'),
#         mpatches.Patch(color='blue', label='Used to force GEMs')] ## include only when plotting both ensembles together
# ax.legend(handles=patches, bbox_to_anchor=(1.45, 1.0), ncol=1)

# plt.savefig('/Users/eultee/Downloads/20250208-RfIntercomp-{}-{}-All.png'.format(basin_label, to_plot), dpi=300)
plt.show()

Double plot, precip and temp, for Indus only.  Keep same axes settings.  Export at higher resolution.  Just for Oslo talk, show precip first and temp below.

In [ ]:
fig, axs = plt.subplots(2,1, sharex=True, figsize=(6,4)) #stack vertically to make sensible shared x

## Plot temperature on first plot
axs[1].axhline(0, color='k', ls=':', lw=0.5)

basin_label = ' '.join([s.capitalize() for s in highlight_basin.split(' ')]) ## manage multi-word basin names

for m in range(len(file_list_T)): ## split by model run
    this_temp = pd.Series(T_by_basin_all[highlight_basin][m])
    this_temp.index=rng
    hist_mean = this_temp['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
    axs[1].plot((this_temp.resample('YE').mean()-hist_mean).rolling(window=20).mean(), ##
           color='grey', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top

for n in range(len(file_list_T_gems)): ## highlight the ones used for GEM forcing
    this_temp = pd.Series(T_by_basin_gems[highlight_basin][n])
    this_temp.index=rng
    hist_mean = this_temp['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
    axs[1].plot((this_temp.resample('YE').mean()-hist_mean).rolling(window=20).mean(), ##
           color='blue', alpha=0.5)

axs[1].set(xlim=(datetime.datetime(2020,12,31),datetime.datetime(2100,12,31)),
           xticks=pd.date_range(datetime.datetime(2020,1,1), periods=5, freq='20YS').tolist(),
           xticklabels=['2020','2040','2060','2080', '2100'],
      ylim=[-1,5], ## Set to values in K rather than percent change
           ylabel = 'Temp change [K]'
      )
axs[0].text(0.05, 0.9, basin_label, transform=axs[0].transAxes)

## Plot precipitation on next one
for m in range(len(file_list)): ## split by model run
    this_precip = pd.Series(P_by_basin_all[highlight_basin][m])
    this_precip.index=rng

    this_precip_dailyavg = this_precip / this_precip.index.days_in_month
    ## handy days_in_month utility
    this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[highlight_basin].squeeze()*1e6)

    hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
    axs[0].plot((this_precip_mmday.resample('YE').mean()-hist_mean).rolling(window=20).mean()/(hist_mean*0.01), ##
           color='grey', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top

for n in range(len(file_list_gems)): ## highlight the ones used for GEM forcing
    this_precip = pd.Series(P_by_basin_gems[highlight_basin][n])
    this_precip.index=rng

    this_precip_dailyavg = this_precip / this_precip.index.days_in_month
    ## handy days_in_month utility
    this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[highlight_basin].squeeze()*1e6)

    hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
    axs[0].plot((this_precip_mmday.resample('YE').mean()-hist_mean).rolling(window=20).mean()/(hist_mean*0.01), ##
           color='blue', alpha=0.5)

axs[0].axhline(0, color='k', lw=0.5)
axs[0].set(xlim=(datetime.datetime(2020,12,31),datetime.datetime(2100,12,31)),
           xticks=pd.date_range(datetime.datetime(2020,1,1), periods=5, freq='20YS').tolist(),
           xticklabels=['2020','2040','2060','2080', '2100'],
           ylim=[-30,71],
           ylabel='Precip change[%]')

# plt.show()
# plt.savefig('/Users/eultee/Downloads/20250208-RfIntercomp-{}-TempPrecip-All.png'.format(basin_label), dpi=300)

---
## Scatter plot to summarize
Try showing GEM-forcing ensemble spread versus CMIP ensemble spread for each basin as a scatter plot?  One idea would be to take the CMIP range in the last 20 years and compare that with the GEM-forcing range over the same period.

Store to a dictionary of DataFrames by basin. Then find the spread over the last 20 years of the century and plot that on a scatter.

In [ ]:
P_df_bybasin = {b: [] for b in basin_name_list}
P_df_bybasin_gem = {b: [] for b in basin_name_list}
T_df_bybasin = {b: [] for b in basin_name_list}
T_df_bybasin_gem = {b: [] for b in basin_name_list}

for j,b in enumerate(basin_name_list):
    temp_dict = {}
    temp_dict_gem = {}
    p_dict = {}
    p_dict_gem = {}
    
    for m in range(len(file_list_T)): ## split by model run
        this_temp = pd.Series(T_by_basin_all[b][m])
        this_temp.index=rng


        hist_mean = this_temp['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        this_T_anom = (this_temp.resample('YE').mean()-hist_mean).rolling(window=20).mean()
        temp_dict[m] = this_T_anom
    T_df_bybasin[b] = pd.DataFrame.from_dict(temp_dict)
    
    for n in range(len(file_list_T_gems)): ## highlight the ones used for GEM forcing
        this_temp = pd.Series(T_by_basin_gems[b][n])
        this_temp.index=rng

        hist_mean = this_temp['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        this_T_anom = (this_temp.resample('YE').mean()-hist_mean).rolling(window=20).mean()
        temp_dict_gem[n] = this_T_anom
    T_df_bybasin_gem[b] = pd.DataFrame.from_dict(temp_dict_gem)


    ## Precipitation
    for m in range(len(file_list)): ## split by model run
        this_precip = pd.Series(P_by_basin_all[b][m])
        this_precip.index=rng
    
        this_precip_dailyavg = this_precip / this_precip.index.days_in_month
        ## handy days_in_month utility
        this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)
    
        hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        this_P_pctchg = (this_precip_mmday.resample('YE').mean()-hist_mean).rolling(window=20).mean()/(hist_mean*0.01)
        p_dict[m] = this_P_pctchg
    P_df_bybasin[b] = pd.DataFrame.from_dict(p_dict)
    
    for n in range(len(file_list_gems)): ## highlight the ones used for GEM forcing
        this_precip = pd.Series(P_by_basin_gems[b][n])
        this_precip.index=rng
    
        this_precip_dailyavg = this_precip / this_precip.index.days_in_month
        ## handy days_in_month utility
        this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)
    
        hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('YE').mean().mean()
        this_P_pctchg = (this_precip_mmday.resample('YE').mean()-hist_mean).rolling(window=20).mean()/(hist_mean*0.01)
        p_dict_gem[n] = this_P_pctchg
    P_df_bybasin_gem[b] = pd.DataFrame.from_dict(p_dict_gem)



In [ ]:
T_df_bybasin['INDUS']['2081-12-31':'2100-12-31'].quantile(q=0.10, axis=1).mean()

In [ ]:
T_lower_quant = np.asarray([T_df_bybasin[b]['2081-12-31':'2100-12-31'].quantile(q=0.25, axis=1).mean()
                 for b in basin_name_list])
T_upper_quant = np.asarray([T_df_bybasin[b]['2081-12-31':'2100-12-31'].quantile(q=0.75, axis=1).mean()
                 for b in basin_name_list])
T_meanmarker_y = np.asarray([T_df_bybasin[b]['2081-12-31':'2100-12-31'].mean(axis=1).mean()
                 for b in basin_name_list])
asym_error_y = [T_meanmarker_y-T_lower_quant, T_upper_quant-T_meanmarker_y]

Tgem_lower_quant = np.asarray([T_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].quantile(q=0.25, axis=1).mean()
                 for b in basin_name_list])
Tgem_upper_quant = np.asarray([T_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].quantile(q=0.75, axis=1).mean()
                 for b in basin_name_list])
Tgem_meanmarker_x = np.asarray([T_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].mean(axis=1).mean()
                 for b in basin_name_list])
asym_error_x = [Tgem_meanmarker_x-Tgem_lower_quant, Tgem_upper_quant-Tgem_meanmarker_x]

In [ ]:
P_lower_quant = np.asarray([P_df_bybasin[b]['2081-12-31':'2100-12-31'].quantile(q=0.25, axis=1).mean()
                 for b in basin_name_list])
P_upper_quant = np.asarray([P_df_bybasin[b]['2081-12-31':'2100-12-31'].quantile(q=0.75, axis=1).mean()
                 for b in basin_name_list])
P_meanmarker_y = np.asarray([P_df_bybasin[b]['2081-12-31':'2100-12-31'].mean(axis=1).mean()
                 for b in basin_name_list])
asym_Perror_y = [P_meanmarker_y-P_lower_quant, P_upper_quant-P_meanmarker_y]

Pgem_lower_quant = np.asarray([P_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].quantile(q=0.25, axis=1).mean()
                 for b in basin_name_list])
Pgem_upper_quant = np.asarray([P_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].quantile(q=0.75, axis=1).mean()
                 for b in basin_name_list])
Pgem_meanmarker_x = np.asarray([P_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].mean(axis=1).mean()
                 for b in basin_name_list])
asym_Perror_x = [Pgem_meanmarker_x - Pgem_lower_quant, 
                 Pgem_upper_quant - Pgem_meanmarker_x]

In [ ]:
fig, axs = plt.subplots(1,2)

axs[0].axline((0,0), slope=1, linestyle='--', color='k')
axs[0].errorbar(Tgem_meanmarker_x, T_meanmarker_y, 
                yerr=asym_error_y, xerr=asym_error_x,
                marker='o', mfc='white', mec='k',
               alpha=0.8)
axs[0].set(xlim=(0,6), ylim=(0, 12), aspect=1)

axs[1].axline((0,0), slope=1, linestyle='--', color='k')
axs[1].axhline(0, linestyle=':', lw=0.5, color='k')
axs[1].axvline(0, linestyle=':', lw=0.5, color='k')
axs[1].errorbar(Pgem_meanmarker_x, P_meanmarker_y, 
                yerr=asym_Perror_y, xerr=asym_Perror_x,
                barsabove=True, capsize=1.0,
                marker='o', mfc='white', mec='k',
               alpha=0.8,
               linestyle='')
axs[1].set(aspect=1)

Something is weird here.  Check the raw values rather than the anomaly / percent change ones?  Report raw spread (max/min over the period) rather than quantiles?

In [ ]:
P_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].max(axis=1).mean()

In [ ]:
T_lower_lim = np.asarray([T_df_bybasin[b]['2081-12-31':'2100-12-31'].min(axis=1).mean()
                 for b in basin_name_list])
T_upper_lim = np.asarray([T_df_bybasin[b]['2081-12-31':'2100-12-31'].max(axis=1).mean()
                 for b in basin_name_list])
T_meanmarker_y = np.asarray([T_df_bybasin[b]['2081-12-31':'2100-12-31'].mean(axis=1).mean()
                 for b in basin_name_list])
asym_error_y = [T_meanmarker_y-T_lower_lim, T_upper_lim-T_meanmarker_y]

Tgem_lower_lim = np.asarray([T_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].min(axis=1).mean()
                 for b in basin_name_list])
Tgem_upper_lim = np.asarray([T_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].max(axis=1).mean()
                 for b in basin_name_list])
Tgem_meanmarker_x = np.asarray([T_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].mean(axis=1).mean()
                 for b in basin_name_list])
asym_error_x = [Tgem_meanmarker_x-Tgem_lower_lim, Tgem_upper_lim-Tgem_meanmarker_x]

In [ ]:
P_lower_lim = np.asarray([P_df_bybasin[b]['2081-12-31':'2100-12-31'].min(axis=1).mean()
                 for b in basin_name_list])
P_upper_lim = np.asarray([P_df_bybasin[b]['2081-12-31':'2100-12-31'].max(axis=1).mean()
                 for b in basin_name_list])
P_meanmarker_y = np.asarray([P_df_bybasin[b]['2081-12-31':'2100-12-31'].mean(axis=1).mean()
                 for b in basin_name_list])
asym_Perror_y = [P_meanmarker_y-P_lower_lim, P_upper_lim-P_meanmarker_y]

Pgem_lower_lim = np.asarray([P_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].min(axis=1).mean()
                 for b in basin_name_list])
Pgem_upper_lim = np.asarray([P_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].max(axis=1).mean()
                 for b in basin_name_list])
Pgem_meanmarker_x = np.asarray([P_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].mean(axis=1).mean()
                 for b in basin_name_list])
asym_Perror_x = [Pgem_meanmarker_x - Pgem_lower_lim, 
                 Pgem_upper_lim - Pgem_meanmarker_x]

In [ ]:
import seaborn as sns

## Organize into dataframes by basin to allow aggregate comparison
## these will be means of the last 20 yr of data
Tdata_all = np.asarray([T_df_bybasin[b]['2081-12-31':'2100-12-31'].mean(axis=0)
                 for b in basin_name_list])
Tdata_gem = np.asarray([T_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].mean(axis=0)
                 for b in basin_name_list])
Pdata_all = np.asarray([P_df_bybasin[b]['2081-12-31':'2100-12-31'].mean(axis=0)
                 for b in basin_name_list])
Pdata_gem = np.asarray([P_df_bybasin_gem[b]['2081-12-31':'2100-12-31'].mean(axis=0)
                 for b in basin_name_list])

Tdata = pd.DataFrame(data=Tdata_all,
                     index=basin_name_list)
Tdata_gem = pd.DataFrame(data=Tdata_gem,
                         index=basin_name_list)
Pdata = pd.DataFrame(data=Pdata_all,
                     index=basin_name_list)
Pdata_gem = pd.DataFrame(data=Pdata_gem,
                         index=basin_name_list)

In [ ]:
Tdata

In [ ]:
fig, axs = plt.subplots(2, sharex=True, sharey=True)

sns.stripplot(Tdata_gem.T,
            ax=axs[0])
axs[0].text(0.99, 0.99, 'GEM forcing', transform=axs[0].transAxes, ha='right', va='top')
axs[0].axhline(0, ls=':', lw=0.5, color='k')
sns.stripplot(Tdata.T,
           ax=axs[1])
axs[1].text(0.99, 0.99, 'CMIP6 archive', transform=axs[1].transAxes, ha='right', va='top')
axs[1].axhline(0, ls=':', lw=0.5, color='k')
axs[1].set(xticklabels=());

fig.supxlabel('Basin')
fig.supylabel(r'End 21st Century $\Delta$T [K rel. 2000-2020]')

In [ ]:
fig, axs = plt.subplots(2, sharex=True, sharey=True)

sns.stripplot(Pdata_gem.T,
            ax=axs[0])
axs[0].text(0.99, 0.99, 'GEM forcing', transform=axs[0].transAxes, ha='right', va='top')
axs[0].axhline(0, ls=':', lw=0.5, color='k')
sns.stripplot(Pdata.T,
           ax=axs[1])
axs[1].text(0.99, 0.99, 'CMIP6 archive', transform=axs[1].transAxes, ha='right', va='top')
axs[1].axhline(0, ls=':', lw=0.5, color='k')
axs[1].set(xticklabels=());

fig.supxlabel('Basin')
fig.supylabel(r'End 21st Century $\Delta$P [% rel. 2000-2020]')

Pair these with a plot comparing the means, so we see whether the sampling is also biased.

In [ ]:
fig,ax = plt.subplots()
ax.axhline(0, ls=':', lw=0.5, color='k')
ax.axvline(0, ls=':', lw=0.5, color='k')
sns.scatterplot(x=Pgem_meanmarker_x, y=P_meanmarker_y, hue=basin_name_list,
               ax=ax, legend=False)
ax.axline((0,0), slope=1, linestyle='--', color='k')
ax.set(aspect=1,
      xlabel=r'Mean $\Delta$P, GEM forcing [%]',
      ylabel=r'Mean $\Delta$P, CMIP6 [%]')

In [ ]:
fig,ax = plt.subplots()
# ax.axhline(0, ls=':', lw=0.5, color='k')
# ax.axvline(0, ls=':', lw=0.5, color='k')
sns.scatterplot(x=Tgem_meanmarker_x, y=T_meanmarker_y, hue=basin_name_list,
               ax=ax, legend=False)
ax.axline((0,0), slope=1, linestyle='--', color='k')
ax.set(aspect=1, xlim=(0, 3.3), ylim=(0,5.2),
      xlabel=r'Mean $\Delta$T, GEM forcing [K]',
      ylabel=r'Mean $\Delta$T, CMIP6 [K]',
      )

## Bar plot with whiskers

Load in glacier area fraction to allow the shading according to glacier area fraction.

In [ ]:
FW_summary_file = '/Users/eultee/Library/CloudStorage/OneDrive-NASA/Data/Runoff-intercomp/Summary-statistics/MASTER_GloGEM.csv'

glacier_cover = pd.read_csv(FW_summary_file, index_col=0).loc[:, 'GlacierAreaFrac'] 
glacier_cover

In [ ]:
from matplotlib import cm
import matplotlib.colors as colors

af_cmap = plt.get_cmap('YlGnBu')
norm = colors.LogNorm(vmin=1e-4, vmax=0.25)

af_colors={}
for b in basin_names:
    afc_normed = norm(glacier_cover[b])
    af_colors[b] = af_cmap(afc_normed)

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))


ax.axhline(0, ls=':', lw=1, color='k')

for i,basin in enumerate(basin_names):
    ax.vlines(x=i, 
              ymin=Pdata.loc[basin].min(),
              ymax=Pdata.loc[basin].max(),
              lw=1.5, color= af_colors[basin]) ## CMIP6 spread

    ax.vlines(x=i, 
              ymin=Pdata_gem.loc[basin].min(),
              ymax=Pdata_gem.loc[basin].max(),
              lw=3.5, color= af_colors[basin]) ##GEM forcing

# ax.axvline(x=0, ls=':') ## confirmed that basin 0 plots at axes pos 0
ax.axvline(x=18.5, ls=':', color='k', lw=0.5) ## Asian basins 0-18
ax.axvline(x=42.5, ls=':', color='k', lw=0.5) ## S. Am. basins 19-42
ax.axvline(x=58.5, ls=':', color='k', lw=0.5) ## N.Am. basins 42-58
ax.axvline(x=59.5, ls=':', color='k', lw=0.5) ## NZ basin 59, then Europe til 74

ax.set(xticks=range(75),
    xticklabels=(),
      xlim=(-0.9, 74.9));

fig.supxlabel('Basin')
fig.supylabel(r'End 21st Century $\Delta$P [% rel. 2000-2020]')

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))


ax.axhline(0, ls=':', lw=1, color='k')

for i,basin in enumerate(basin_names):
    ax.vlines(x=i, 
              ymin=Tdata.loc[basin].min(),
              ymax=Tdata.loc[basin].max(),
              lw=1.5, color= af_colors[basin]) ## CMIP6 spread

    ax.vlines(x=i, 
              ymin=Tdata_gem.loc[basin].min(),
              ymax=Tdata_gem.loc[basin].max(),
              lw=3.5, color= af_colors[basin]) ##GEM forcing

# ax.axvline(x=0, ls=':') ## confirmed that basin 0 plots at axes pos 0
ax.axvline(x=18.5, ls=':', color='k', lw=0.5) ## Asian basins 0-18
ax.axvline(x=42.5, ls=':', color='k', lw=0.5) ## S. Am. basins 19-42
ax.axvline(x=58.5, ls=':', color='k', lw=0.5) ## N.Am. basins 42-58
ax.axvline(x=59.5, ls=':', color='k', lw=0.5) ## NZ basin 59, then Europe til 74

ax.set(xticks=range(75),
    xticklabels=(),
      xlim=(-0.9, 74.9));

fig.supxlabel('Basin')
fig.supylabel(r'End 21st Century $\Delta$T [K rel. 2000-2020]')